In [ ]:
#####################################
##########  训练landmarks  ##########

In [ ]:
#@title 一、数据准备
# # 1、上传数据集
# cd LangXin
unzip data.zip
# rm -rf data.zip

# # 2、将视频转化为16000赫兹，单声道
python 16000.py

# # 3、将视频切割成2-5秒一段
python split_video.py

# # 4、生成 train.txt 、test.txt文档
python  sample.py

# # 5、将train.txt 、test.txt按照顺序排列
python  123456.py

# # 6、将视频转化为16000赫兹，单声道
python 16000-more.py

In [ ]:
#@title 二、训练数据预处理
cd preprocess
# cd preprocess
# # 1、音频处理
python preprocess_audio.py --data_root ../data --out_root ../data_audio

# # 2、 视频处理
# # 确定好人脸 pading 像素值,默认是5。
cd  preprocess
python preprocess_video.py --dataset_video_root ../data --output_sketch_root  ../data_sketch --output_face_root  ../data_face --output_landmark_root  ../data_landmarks

In [ ]:
# #@title 三、训练数据预处理
# # 1、训练landmarks
# cd LangXin
python train_landmarks_generator.py --pre_audio_root  ./data_audio  --landmarks_root ./data_landmarks  
python train_landmarks_generator.py --pre_audio_root  ./data_audio  --landmarks_root ./data_landmarks   --load_checkpoint ./checkpoints/landmark_generation/Pro_landmarkT5_d512_fe1024_lay4_head4/landmarkT5_d512_fe1024_lay4_head4_epoch_2426_checkpoint_step001150000.pth 

# # 如果报错的话就将
# # shuffle=True, 改成
# # shuffle=False,
# ########################
# # 如果再报错，将
# # num_workers = 4
# # batch_size =12  # # 512

In [ ]:
# # 2、训练renderer渲染器
# cd LangXin
python train_video_renderer.py --sketch_root ./data_sketch --face_img_root ./data_face  --audio_root ./data_audio


# # 如果报错的话就将
# # shuffle=True, 改成
# # shuffle=False,
# ########################
# # 如果再报错，将
# # num_workers = 4
# # batch_size =12  # # 512

In [ ]:
#@title 四、进行推理准备，将自己训练好的权重复制并放在下面路径
cp XXXXXXXXX     ./test/checkpoints


In [ ]:
#@title 五  运行训练后的模型推理 
# 改代码inference_single.py  line 34 fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False, device='cuda')
cd /content/LangXin
python inference_single.py --input ./test/555.mp4  --audio ./test/555.wav  --landmark_gen_checkpoint_path  ./test/new-checkpoints/new-landmarkT5_d512_fe1024_lay4_head4_epoch_17000_checkpoint_step000850000.pth   --renderer_checkpoint_path  ./test/new-checkpoints/new-renderer_T1_ref_N3_epoch_661_checkpoint_step000354000.pth

In [ ]:
# 查看文件夹下面， 每个子文件夹下面的文件数量
for d in */ ; do echo "$d"; find "$d" -maxdepth 1 -type f | wc -l; done

# 查看文件大小
du -sh 300

#  查看音频采样
ffmpeg -i audio.wav


In [ ]:
# 跨区域拷贝数据
# 拷贝到那个区的项目
ssh -p 17302 root@connect.westb.seetacloud.com
# 从此项目拷贝到目标项目中
scp -rP 17302 /root/landmarkT5_d512_fe1024_lay4_head4_epoch_714_checkpoint_step000005000.pth  root@connect.westb.seetacloud.com:/root/autodl-tmp

In [ ]:
ssh -p 44125 root@connect.westb.seetacloud.com
scp -rP 44125 /root/autodl-tmp/300-000   root@connect.westb.seetacloud.com:/root/autodl-tmp

In [ ]:
# tensorboard显示loss
tensorboard --port 7860 --logdir /root/autodl-tmp/LangXin/tensorboard_runs/Project_landmarkT5_d512_fe1024_lay4_head4